In [20]:
import cv2
import numpy as np
from keras.models import load_model
import os

In [21]:
browser_name = "SequenceBrowser"
model_file = r"j:\SavedModels\model_2019-06-11_00-12-58.h5"
image_name = "Video_Video"
output_folder = r"j:\Temp"

classes = ['A', 'C', 'None', 'R', 'S']

In [22]:
browser_node = slicer.util.getFirstNodeByName(browser_name, className='vtkMRMLSequenceBrowserNode')
if browser_node is None:
    logging.error("Could not find browser node: {}".format(browser_name))
    raise

image_node = slicer.util.getFirstNodeByName(image_name)
if image_node is None:
    logging.error("Could not find image node: {}".format(image_name))
    raise

n = browser_node.GetNumberOfItems()

print("Sequence browser ID: {}".format(browser_node.GetID()))
print("Image node ID:       {}".format(image_node.GetID()))
print("Number of images:    {}".format(n))

Sequence browser ID: vtkMRMLSequenceBrowserNode1
Image node ID:       vtkMRMLStreamingVolumeNode1
Number of images:    45


In [23]:
try:
    model = load_model(model_file)
except:
    print("Could not load Keras model froim {}".format(model_file))
    raise

model_input_size = model.layers[0].input_shape[1]

print("Model input size: {}".format(model_input_size))

Model input size: 64


In [24]:
predictions = np.zeros([n, len(classes)])
resized_images = np.zeros([n, model_input_size, model_input_size, 3])

In [25]:
browser_node.SelectFirstItem()

for i in range(n):
    image = image_node.GetImageData()
    shape = list(image.GetDimensions())
    shape.reverse()
    components = image.GetNumberOfScalarComponents()
    if components > 1:
        shape.append(components)
        shape.remove(1)
    input_array = vtk.util.numpy_support.vtk_to_numpy(image.GetPointData().GetScalars()).reshape(shape)
    resized_input_array = cv2.resize(input_array, (model_input_size, model_input_size))
    resized_input_array = resized_input_array / (resized_input_array.max())
    # resized_input_array = np.flip(resized_input_array, axis=0)
    resized_images[i, :, :, :] = resized_input_array[:, :, :]
    
    resized_input_array = np.expand_dims(resized_input_array, axis=0)
    prediction = model.predict(resized_input_array)
    predictions[i, :] = prediction[:]
    
    maxPredictionIndex = prediction.argmax()
    
    browser_node.SelectNextItem()
    slicer.app.processEvents()

In [27]:
import os

images_fullname = os.path.join(output_folder, "resized_images")
pred_fullname = os.path.join(output_folder, "predictions")
np.save(images_fullname, resized_images)
np.save(pred_fullname, predictions)